In [1]:
import os
import PIL
import dlib
# import matplotlib.pyplot as plt
import datetime
import numpy as np

from sklearn.metrics import confusion_matrix
# import seaborn as sns
import pandas as pd

In [2]:
import torch
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split, SubsetRandomSampler
import torch.optim as optim

import tensorboardX

In [3]:
torch.manual_seed(0)
np.random.seed(0)

In [5]:
epochs = 50
batch_size = 16
lr = 1e-4

In [6]:
logname = f'conv2d_{batch_size}_{epochs}_{lr}_{datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}'

if not os.path.exists(os.path.join('models', logname)):
    os.makedirs(os.path.join('models', logname))

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [8]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [9]:
data_root = 'images/dataset'
save_path = 'images/cropped_faces'

In [10]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [11]:
class FaceDataset(Dataset):
    def __init__(self, n_classes=4, root_dir=save_path, transform=transform):
        self.root_dir = root_dir
        self.transform = transform
        self.files = os.listdir(self.root_dir)
        # self.labels = [int(file.split('_')[0]) for file in self.files]
        
        self.n_classes = n_classes
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.files[idx])
        img = PIL.Image.open(img_name)
        img = self.transform(img)
        label = int(self.files[idx].split('_')[0])
        label = label if self.n_classes == 4 else label // 2
        return img, label

In [12]:
dataset = FaceDataset(n_classes=4)
train_data = dataset

# len_data = len(dataset)
# train_size = int(0.8 * len_data)
# test_size = len_data - train_size

# train_data, test_data = random_split(dataset, [train_size, test_size])
# test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

# print('Train:', len(train_data), 'Test:', len(test_data))

In [13]:
class Model(nn.Module):
    def __init__(self, n_classes):
        super(Model, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=0)
        self.conv3 = nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1)
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(16 * 15 * 15, n_classes)
        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.4)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        # (input - f + 2p) / s + 1
    def forward(self, x):
        x = self.relu(self.conv1(x))  # (256 - 5 + 1) / 2 + 1 = 128  [16, 128, 128]
        x = self.maxpool(x)  # [16, 64, 64]
        x = self.relu(self.conv2(x))  # (64 - 5) + 1 = 60 [32, 60, 60]
        x = self.maxpool(x)  # [32, 30, 30]
        x = self.relu(self.conv3(x))  # [16, 30, 30]
        x = self.maxpool(x)  # [16, 15, 15]
        x = self.dropout1(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.dropout2(x)
        x = self.softmax(x)
        return x

In [14]:
writer = tensorboardX.SummaryWriter(logdir=f'runs/{logname}'+ datetime.datetime.now().strftime('%Y%m%d%H%M%S'))

In [15]:
def train(model, train_loader, optimizer, criterion, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        writer.add_scalar('train_loss', loss.item(), i*epoch)
        writer.add_scalar('train_accuracy', 100*correct/total, i*epoch)
        
        if i % 10 == 99:
            print(f'[{i+1}, {running_loss/100:.3f}]')
            running_loss = 0.0
            correct = 0
            total = 0
        
    train_loss = running_loss/len(train_loader)
    train_accuracy = 100*correct/total
    print(f'Training {epoch+1}/{epochs}:\tTrain loss: {train_loss:.3f},\tTrain acc: {train_accuracy:.3f}')
    
    return train_loss, train_accuracy

In [16]:
y_pred = []
y_true = []
    
def test(model, test_loader, criterion):
    model.eval()
    
    total_loss = 0
    
    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            y_pred.extend(predicted.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
            
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
    test_loss = total_loss / len(test_loader)
    test_acc = 100 * sum([1 for i in range(len(y_pred)) if y_pred[i] == y_true[i]]) / len(y_true)
    print(f'Test:\tTest loss: {test_loss:.3f},\tTest acc: {test_acc:.3f}')
    
    writer.add_scalar('test_accuracy', test_acc)
    writer.add_scalar('test_loss', test_loss)
    
    return test_loss, test_acc

In [17]:
# def train_val_curve(epoch, fold, train_losses, train_accuracies, val_losses, val_accuracies):
#     fig, ax = plt.subplots(2, 1, figsize=(10, 10))
    
#     ax[0].plot(range(epochs*fold), train_losses, label='train')
#     ax[0].plot(range(epochs*fold), val_losses, label='val')
#     ax[0].set_title('Loss')
#     ax[0].legend()
    
#     ax[1].plot(range(epochs*fold), train_accuracies, label='train')
#     ax[1].plot(range(epochs*fold), val_accuracies, label='val')
#     ax[1].set_title('Accuracy')
#     ax[1].legend()
    
#     plt.show()
    
#     writer.add_figure('train_val_curve', fig, epoch*fold)

In [18]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, shuffle=True, random_state=0)

In [19]:
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform(m.weight.data)

In [20]:
train_losses_folds, train_accs_folds = [], []
test_losses_folds, test_accs_folds = [], []

In [21]:
def run():
    for fold, (train_idx, test_idx) in enumerate(kfold.split(train_data)):
        print(f'Fold {fold+1}/{5}')
        
        train_subset = SubsetRandomSampler(train_idx)
        test_subset = SubsetRandomSampler(test_idx)
        train_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_subset)
        test_loader = DataLoader(train_data, batch_size=batch_size, sampler=test_subset)
        
        model = Model(dataset.n_classes).to(device)
        model.apply(weights_init)
        
        optimizer = optim.Adam(model.parameters(), lr=lr)
        criterion = nn.CrossEntropyLoss()
        
        train_losses = []
        train_accuracies = []
        
        for epoch in range(epochs):
            
            train_loss, train_acc = train(model, train_loader, optimizer, criterion, epoch)
            
            train_losses.append(train_loss)
            train_accuracies.append(train_acc)
            
        train_losses_folds.append(sum(train_losses) / len(train_losses))
        train_accs_folds.append(sum(train_accuracies) / len(train_accuracies))
        print(f'Train {fold+1}/{5}:\tTrain loss: {train_losses_folds[-1]:.3f},\tTrain acc: {train_accs_folds[-1]:.3f}')
        
        test_loss, test_acc = test(model, test_loader, criterion)
        print(f'Test {fold+1}/{5}:\tTest loss: {test_loss:.3f},\tTest acc: {test_acc:.3f}')
        
        test_losses_folds.append(test_loss)
        test_accs_folds.append(test_acc)
                
    final_train_loss = sum(train_losses_folds) / len(train_losses_folds)
    final_train_acc = sum(train_accs_folds) / len(train_accs_folds)
    
    final_test_loss = sum(test_losses_folds) / len(test_losses_folds)
    final_test_acc = sum(test_accs_folds) / len(test_accs_folds)
    
    return final_train_loss, final_train_acc, final_test_loss, final_test_acc

In [22]:
train_loss, train_acc, train_loss, train_acc = run()

Fold 1/5


C:\Users\pomat\AppData\Local\Temp\ipykernel_64424\1855755017.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight.data)


Training 1/100:	Train loss: 1.369,	Train acc: 32.605
Training 2/100:	Train loss: 1.355,	Train acc: 35.157
Training 3/100:	Train loss: 1.338,	Train acc: 37.267
Training 4/100:	Train loss: 1.326,	Train acc: 38.199
Training 5/100:	Train loss: 1.312,	Train acc: 40.088
Training 6/100:	Train loss: 1.307,	Train acc: 40.677
Training 7/100:	Train loss: 1.293,	Train acc: 42.174
Training 8/100:	Train loss: 1.278,	Train acc: 44.112
Training 9/100:	Train loss: 1.272,	Train acc: 45.461
Training 10/100:	Train loss: 1.263,	Train acc: 46.246
Training 11/100:	Train loss: 1.253,	Train acc: 47.792
Training 12/100:	Train loss: 1.247,	Train acc: 48.307
Training 13/100:	Train loss: 1.244,	Train acc: 48.577
Training 14/100:	Train loss: 1.240,	Train acc: 49.117
Training 15/100:	Train loss: 1.227,	Train acc: 49.828
Training 16/100:	Train loss: 1.225,	Train acc: 50.466
Training 17/100:	Train loss: 1.226,	Train acc: 50.074
Training 18/100:	Train loss: 1.223,	Train acc: 50.638
Training 19/100:	Train loss: 1.220,	T

In [ ]:
# plt.figure(figsize=(10, 5))
# plt.plot(train_accuracies, label='Train Accuracy')
# plt.plot(val_accuracies, label='Val Accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.title('Train and Val Accuracy')
# plt.legend()
# plt.show()

# plt.figure(figsize=(10, 5))
# plt.plot(train_losses, label='Train Loss')
# plt.plot(val_losses, label='Val Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Train and Val Loss')
# plt.legend()
# plt.show()

In [ ]:
classes = ['Spring', 'Summer', 'Fall', 'Winter'] if dataset.n_classes == 4 else ['Warm', 'Cool']

In [ ]:
cm = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cm, index=classes, columns=classes)

In [ ]:
plt.figure(figsize=(10, 7))
sns.heatmap(df_cm, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')
writer.add_figure('confusion_matrix', plt.gcf())